## Expected Turnout

To predict turnout we have a number of different models that we can use. In the cells below you can select a turnout model and tune the parameter for statewide expected turnout. 

---
Our different models are as follows:

1. `exp_16` - models precinct level turnout with the same turnout numbers as 2016
2. `exp_08` - models precinct level turnout with the same turnout numbers as 2008
3. `exp_avg` - models precinct level turnout with the average of the precinct level turnout numbers from 2016 and 2008
4. `exp_percent_16` - models precinct level turnout using the percent of statewide vote per precicnt in 2016 and a parameter for statewide turnout (`expected_statewide_turnout`) that can be adjusted.
5. `exp_percent_08` - models precinct level turnout using the percent of statewide vote per precicnt in 2008 and a parameter for statewide turnout (`expected_statewide_turnout`) that can be adjusted.
6. `exp_percent_avg` - models precinct level turnout using the average of the percents of statewide vote per precicnt in 2016 and 2008, and a parameter for statewide turnout (`expected_statewide_turnout`) that can be adjusted.7.
7. `overall_avg` - models precinct level turnout using the average of `exp_16`, `exp_08`, `exp_percent_16`, and `exp_percent_08`.

By default `expected_statewide_turnout` is set to 300,000.




In [112]:
def exp_16 (row):
    return row['count16']

def exp_08 (row):
    return row['count08']

def exp_avg (row):
    return (exp_16(row) + exp_08(row)) / 2

def exp_percent_16 (row):
    return (row['count16'] / statewide_turnout_16) * expected_statewide_turnout

def exp_percent_08 (row):
    return (row['count08'] / statewide_turnout_08) * expected_statewide_turnout

def exp_percent_avg (row):
    return (exp_percent_16(row) + exp_percent_08(row)) / 2

def overall_avg (row):
    return (exp_16(row) + exp_08(row) + exp_percent_16(row) + exp_percent_08(row)) / 4

In [113]:
### SET THE TURNOUT MODEL YOU WANT TO USE ###
turnout_model = overall_avg

### SET THE EXPECTED STATEWIDE TURNOUT IF YOUR MODEL DEPENDS ON IT ###
expected_statewide_turnout = 300000

## Predicted Turnout (Based on IDs)

To predict the number of caucus goers for different candidates in individual precincts we assigns weights to each ID we have. 

First, we weigh each ID based on their response to the Caucus Plan survey question. These values can be set in Caucus Plan Weights. We also include a default value for folks with no response to the Caucus Plan survey question.

We then also weight IDs based on their strength. For Warren IDs there are two strengths "committed" and "lean." All other candidate IDs are set to a single strength weight. These values can be set in Strength Weights

Finaly, we assume a flake rate for our weighted sum only for Warren turnout. 

---
### Caucus Plan Weights

The following variable names correspond to the different answers to the caucus plan question and their default weights are shown.

* `yes_ip`,      Yes - In Person, Default Weight: 0.9
* `yes_vc`,     Yes - Virtual Caucus, Default Weight: 0.9
* `maybe_ip`,    Maybe - In Person, Default Weight: 0.6
* `maybe_vc`,    Maybe - Virtual Caucus, Default Weight: 0.6
* `not_caucus`,  Not Caucusing, Default Weight: 0.1
* `GOP`,         GOP, Default Weight: 0
* `default`,      No Survey Question Response, Default Weight: 0.8


---
### Stength Weights

The following variable names correspond to the different strengths for an ID and thier defualt weights are shown.

* `committed_id`, Default Weight: 1
* `lean_id`, Default Weight: 0.5
* `other_candidate_id`, Default Weight: 1.3

---
### Flake Rate

The `flake_rate` variable can be set to the desired flake rate. By defualt it is set to 85%. Note that the variable is set to the percenate of people who _do_ show up. The flake rate is only applied to calculate expected turnout for EW.

In [114]:
### SET THE CAUCUS PLAN WEIGHTS HERE ###
yes_ip = 0.9
yes_vc = 0.9
maybe_ip = 0.6
maybe_vc = 0.6
not_caucus = 0
GOP = 0
default = 0.9

### SET THE STRENGTH WEIGHTS HERE ###
committed_id = 1
lean_id = 0.5
other_candidate_id = 1.3

### SET THE DESIRED FLAKE RATE HERE ###
flake_rate = 0.85

## Single Delegate Districts

In calculating 'Distance to Next Delegate' for single delegate precincts we use 50% + 1 as the distance and the viability threshold. This percent (50%) can be changed in a variable.

---

Set the `single_delegate_percent` varaible to adjust the 50% threshold for single delgate precicnts. 

In [115]:
### SET THE DESIRED VIABILITY WEIGHTS HERE ###
viability_percent = 0.7
single_delegate_percent = 0.5

## Predicted Delegates and Distance to Next Delegate

Predicting the number of delegates we expect for EW and calculating the distance to next delegate require an assumption about turnout. By default we use the value in the expected turnout column. However, when the turnout we predict from IDs is higher than this value, we switch to using the turnout from IDs as the value in our model. This can be done in one of two ways: (1) we can use the total expected turnout from our IDs or (2) we can use the total expeted turnout from only our IDs of candidates we expect to be viable.

---

Set the `only_viable_ids` variable to `True` to use only the IDs from candidates we expect to be viable. By default this value is set to `False`.

In [116]:
### SET THE DESIRES ID TURNOUT ASSUMPTION HERE ###
only_viable_ids = False

In [117]:
import numpy as np
import pandas as pd
import math
import civis
import json
import datetime
import sys

In [118]:
### IMPORT CIVIS DATA ###

# Setup API client
client = civis.APIClient()

# Import precinct data
to_drop = ['clinton_16', 'hubbell_18', 'clinton_hubbell_sum', 'reporting_multiplier','percent_of_statewide_vote']
sql = "SELECT * FROM analytics_ia.precinct_data"
precinct_data = civis.io.read_civis_sql(sql, "Warren for MA", use_pandas=True, client=client)
precinct_data.drop(to_drop, inplace=True, axis=1)

# Import caucus history data 
sql = "SELECT van_precinct_id, SUM(case when caucus_attendee_2016 = 1 then 1 else 0 end) count16, SUM(case when caucus_attendee_2008 = 1 then 1 else 0 end) count08 FROM phoenix_caucus_history_ia.person_caucus_attendance ca LEFT JOIN phoenix_ia.person p ON ca.person_id = p.person_id GROUP BY van_precinct_id"
caucus_history = civis.io.read_civis_sql(sql, "Warren for MA", use_pandas=True, client=client)

# Import Organizer Turfs
sql = "select van_precinct_id, fo_name from vansync_ia.turf"
turfs = civis.io.read_civis_sql(sql, "Warren for MA", use_pandas=True, client=client)
turfs.set_index('van_precinct_id', inplace=True)

# Import Caucus Plan Response and first choice ID data
sql = "SELECT choice_1.van_precinct_id, caucus_plan.survey_response_name, choice_1.survey_response_name, count(*) FROM(SELECT van_precinct_id, person_id, myv_van_id, myc_van_id, survey_response_name FROM analytics_ia.vansync_responses WHERE mrr_all = 1 AND survey_question_name = '1st Choice Caucus') choice_1 LEFT JOIN(SELECT van_precinct_id, person_id, myv_van_id, myc_van_id, survey_response_name FROM analytics_ia.vansync_responses WHERE mrr_all = 1 AND survey_question_name = 'Caucus Plan') caucus_plan ON (choice_1.myv_van_id = caucus_plan.myv_van_id OR choice_1.myc_van_id = caucus_plan.myc_van_id) GROUP BY 1,2,3"
cube_load = civis.io.read_civis_sql(sql, "Warren for MA", use_pandas=True, client=client)

In [119]:
### Format Imported Data ###

# Rename columns in precicnt_data dataframe
precinct_data.rename(index=str, columns={"congressional_district": "Congressional District", 
                                    "precinct_id": "Precinct ID", 
                                    "county": "County",
                                    "precinct_code": "Precinct Code",
                                    "sos_precinct_name": "Sec. State Precinct Name",
                                    "delegates_to_county_conv": "Delegates to County Conv",
                                    "state_delegate_equivalence_sde": "State Delegate Equivalence (SDE)"}, inplace=True)

# Set dtype for columns to float
cols = first_choice.columns.drop('survey_response_name')
first_choice[cols] = first_choice[cols].astype(np.float32)
# Pivot on van_precinct_id
fc = first_choice.pivot(index='van_precinct_id', columns='survey_response_name', values='count')
# Reset dtype for columns to float
cols = fc.columns
fc[cols] = fc[cols].astype(np.float32)

# Pivot cube
cube  = cube_load
cube['survey_response_name'].fillna('No Response', inplace=True)
cube = cube_load.pivot_table(index=['van_precinct_id'], columns=['survey_response_name','survey_response_name col2'])
cube.columns = cube.columns.droplevel()
cube = cube.fillna(0)

In [121]:
# Create new dataframe, df, of historical caucus data and precicnt data
df = pd.merge(precinct_data, caucus_history, left_on='Precinct ID', right_on='van_precinct_id')
# Set the index of the new df
df.set_index('Precinct ID', inplace=True)

In [122]:
### ADD EXPECTED TURNOUT TO DF ###

# First, calculate statewide turnouts for certain models
statewide_turnout_16 = caucus_history['count16'].sum()
statewide_turnout_08 = caucus_history['count08'].sum()
# Then apply selected model to each row
df['Expected Turnout'] = df.apply(turnout_model, axis=1)
df['Expected Turnout'] = df['Expected Turnout'].apply(lambda row: round(row, 0))
# Remove historical caucus data after turnout calculations are complete
columns = ['van_precinct_id', 'count16', 'count08']
df.drop(columns, inplace=True, axis=1)

In [123]:
### ADD SDE PER PERSON TO DF ###

# Calculate SDE per person based on turnout model
df['SDE per Person'] = df.apply(lambda row : row['State Delegate Equivalence (SDE)'] / row['Expected Turnout'], axis=1)
# Round to 3 digits
df['SDE per Person'] = df['SDE per Person'].apply(lambda row: round(row, 3))

In [124]:
### Add COMMITTED AND LEAN WARREN TO DF ###

# Merge Committed Warren fc to df
#df = pd.merge(df, fc[['Committed Warren']], how="left", left_index=True, right_index=True)
# Merge Lean Warren fc to df
#df = pd.merge(df, fc[['Lean Warren']], how="left", left_index=True, right_index=True)

In [125]:
### ADD COMMITTED WARREN, LEAN WARREN, AND EXPECTED WARREN TUNROUT AND TO DF ###

# Get values for each precinct
def turnout(row):
    # expected turnout
    turnout = 0
    try:
        turnout += cube.loc[row.name, ('Yes - In Person', 'Committed Warren')] * yes_ip * committed_id
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('Maybe - In Person', 'Committed Warren')] * maybe_ip * committed_id
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('Not Caucusing', 'Committed Warren')] * not_caucus * committed_id
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('No Response', 'Committed Warren')] * yes_ip * committed_id
    except:
        pass
    
    try:
        turnout += cube.loc[row.name, ('Yes - In Person', 'Lean Warren')] * yes_ip * lean_id
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('Maybe - In Person', 'Lean Warren')] * maybe_ip * lean_id
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('Not Caucusing', 'Lean Warren')] * not_caucus * lean_id
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('No Response', 'Lean Warren')] * yes_ip * lean_id
    except:
        pass

    # Committed Warren across all caucus plan catagories
    committed_warren = cube.loc[row.name, (slice(None), 'Committed Warren')].sum()
    
    # Lean Warren across all caucus plan catagories
    lean_warren = cube.loc[row.name, (slice(None), 'Lean Warren')].sum()
    
    return committed_warren, lean_warren, turnout * flake_rate
    
# Add values for each precinct to DF
df[['Committed Warren', 'Lean Warren', 'Expected Warren Turnout']] = cube.apply(turnout, axis=1, result_type="expand")
# Round to whole numbers
df['Expected Warren Turnout'] = df['Expected Warren Turnout'].apply(lambda row: round(row, 0))
# Fill NAn values with 0s
df = df.fillna(0)

In [127]:
### ADD ID TURNOUT TO DF ###

# Get list of candidates from cube
to_drop = ['Committed Warren', 'Lean Warren', 'GOP', 'Other Dem', 'Undecided', 'Refused to say']
candidates = cube.columns.get_level_values(1).unique().drop(to_drop).to_list()

# Build a table of expected turnout for each candidate
turnout_by_candidate = pd.DataFrame(index=df.index, columns=candidates)
turnout_by_candidate = turnout_by_candidate.fillna(0)

def other_candidate_turnout(row):
    turnout = 0
    try:
        turnout += cube.loc[row.name, ('Yes - In Person', str(candidate))] * yes_ip
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('Maybe - In Person', str(candidate))] * maybe_ip
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('Not Caucusing', str(candidate))] * not_caucus 
    except:
        pass
    try:
        turnout += cube.loc[row.name, ('No Response', str(candidate))] * yes_ip
    except:
        pass
    return turnout * other_candidate_id

# Populate the table
for candidate in candidates:
    turnout_by_candidate[candidate] = cube.apply(other_candidate_turnout, axis=1)

# Add total ID turnout to df
df['Other Canidate ID Turnout'] = round(turnout_by_candidate.sum(axis=1), 0)

In [128]:
### ADD VIABILITY THRESHOLD TO DF ###

# Function that maps number of county convention delegates to viability formula
def viability_threshold(num_del):
    if num_del == 0:
        return 0.0
    elif num_del == 1:
        return 0.0
    elif num_del == 2:
        return 0.25
    elif num_del == 3:
        return 1 / 6
    else:
        return 0.15
    
# Assign viablity thresholds based on the number of County Convention delegates and the turnout model
df['Viability Threshold'] = df.apply(lambda row : math.ceil(max(row['Expected Turnout'], row['Other Canidate ID Turnout']) * viability_threshold(row['Delegates to County Conv'])), axis=1).astype(np.float32)

In [129]:
### ADD WARREN VIABILITY TO DF ###

# Calculate whether or not EW is viable for each precinct
df['Warren Viable'] = df.apply(lambda row: row['Viability Threshold'] <= row['Expected Warren Turnout'] and row['Expected Warren Turnout'] != 0, axis=1)

In [130]:
### ADD OTHER VIABLE AND VIABLE TURNOUT TO DF ###

# count the number of other viable candidates in each precinct
# calculate the expected turnout of only the other candidates that are viable
def count_viable(row):
    num_viable = 0
    viable_turnout = 0
    for candidate in candidates:
        if (df.loc[row.name]["Viability Threshold"] <= row[candidate] and row[candidate] != 0):
            num_viable += 1
            viable_turnout += row[candidate]
    return num_viable, round(viable_turnout, 0)

df[["Other Viable Candidates", "Other Candidate Viable Turnout"]] = turnout_by_candidate.apply(count_viable, axis=1, result_type="expand")

In [131]:
### ADD EXPECTED WARREN DELEGATES TO DF ####

# Calculate expected number of warren delegates based on exprected warren turnout
def expected_dels (row):
    et = row['Expected Turnout']
    ew = row['Expected Warren Turnout']
    if only_viable_ids:
        oc = row['Other Candidate Viable Turnout']
    else:
        oc = row['Other Canidate ID Turnout']
    id_turnout = ew + oc
    num_del = row['Delegates to County Conv']
    
    # Return 0 if not viable
    if (not row['Warren Viable']):
        return 0
    
    exp_del = num_del * (ew) / max(et, id_turnout)
    if (exp_del % 1) >= 0.5:
        return math.ceil(exp_del)
    else:
        return math.floor(exp_del)

df['Expected Warren Delegates'] = df.apply(expected_dels, axis=1)    

In [132]:
### ADD DISTANCE TO VIABILITY TO DF ###

def distance_to_viability (row):
    et = row['Expected Turnout']
    ew = row['Expected Warren Turnout']
    if only_viable_ids:
        oc = row['Other Candidate Viable Turnout']
    else:
        oc = row['Other Canidate ID Turnout']
    num_other = row['Other Viable Candidates']
    id_turnout = ew + oc
    num_del = row['Delegates to County Conv']
    
    n = row['Expected Warren Delegates'] + 0.5
    
    # Lee County
    if num_del == 0:
        return None
    # One delegate precinct
    if num_del == 1:
        return None
    # More than one delegate precicnt
    else:
        return math.ceil(row['Viability Threshold'] - ew)

df['Distance to Viability'] = df.apply(distance_to_viability, axis=1)      

In [133]:
### ADD DISTANCE TO NEXT DELEGATE TO DF ###

def distance_to_next_delegate (row):
    et = row['Expected Turnout']
    ew = row['Expected Warren Turnout']
    if only_viable_ids:
        oc = row['Other Candidate Viable Turnout']
    else:
        oc = row['Other Canidate ID Turnout']
    num_other = row['Other Viable Candidates']
    id_turnout = ew + oc
    num_del = row['Delegates to County Conv']
    
    n = row['Expected Warren Delegates'] + 0.5
    
    # Lee County
    if num_del == 0:
        return None
    
    # One delegate precinct
    elif num_del == 1:
        # Distance to 50% + 1 of expected turnout or id_turnout (whichever is higher)
        return math.ceil((max(et, id_turnout) * (0.5)) + 1 - ew)
    
    # More than one delegate precicnt
    else:
        
        # If we are not yet viable return the distance to viability
        if (not row['Warren Viable']):
            return math.ceil(row['Viability Threshold'] - ew)
        
        else:
        
            # When there are more viable candidates than there are delegates
            if (num_other + 1 > num_del):
                return -1

            # Calculate distance to next assuming expected turnout
            dist_to_15 = math.ceil(((n * et) - (num_del * ew)) / num_del)

            # If the total is still less than or equal to expected turnout
            if (id_turnout + dist_to_15 <= et):
                return dist_to_15

            # Otherwise calculate distance to next with id_turnout
            else:
                return math.ceil(((n * (ew + oc)) - (num_del * ew)) / (num_del - n))
            
df['Distance to Next Delegate'] = df.apply(distance_to_next_delegate, axis=1)      

In [134]:
### FORMAT DF ###

# Drop columns only used for internal calculations
df.drop(['Other Candidate Viable Turnout'], inplace=True, axis=1)
# Add organizer turfs
df = pd.merge(df, turfs[['fo_name']], how='left', left_index=True, right_index=True)
# Sort columns
df.sort_values(['fo_name', 'Distance to Next Delegate', 'State Delegate Equivalence (SDE)'], inplace=True)

In [135]:
### CREATE COUNTY DATEFRAME ###

# Calculate county level sums
county_totals = df.groupby(['County']).sum()
# Drop unnessecary columns
to_drop = ['Congressional District', 'SDE per Person','Warren Viable', 'Other Viable Candidates']
county_totals.drop(to_drop, inplace=True, axis=1)
# Find county level counts
county_totals['Total Precincts']  = df.groupby('County').size()
county_totals['Viable Precincts']  = df.groupby('County')['Warren Viable'].apply(lambda x: x[x == True].count())
# Find county level means
county_means = df.groupby(['County']).mean()
# Rename means
county_means.rename(index=str, columns={"Viability Threshold": "Mean Viability Threshold", 
                                        "Distance to Next Delegate": "Mean Distance to Next Delegate",}, inplace=True)
# Merge means to totals
county_totals = pd.merge(county_totals, county_means[["Mean Viability Threshold", "Mean Distance to Next Delegate"]], how='left', left_index=True, right_index=True)

In [ ]:
# Round data
df['State Delegate Equivalence (SDE)'] = df['State Delegate Equivalence (SDE)'].apply(lambda row: round(row, 3))

county_totals['State Delegate Equivalence (SDE)'] = county_totals['State Delegate Equivalence (SDE)'].apply(lambda row: round(row, 0))
county_totals['Mean Viability Threshold'] = county_totals['Mean Viability Threshold'].apply(lambda row: round(row, 3))

In [ ]:
county_totals

,Delegates to County Conv,State Delegate Equivalence (SDE),Expected Turnout,Committed Warren,Lean Warren,Expected Warren Turnout,Other Canidate ID Turnout,Viability Threshold,Expected Warren Delegates,Distance to Viability,Distance to Next Delegate,Total Precincts,Viable Precincts,Mean Viability Threshold,Mean Distance to Next Delegate
County,,,,,,,,,,,,,,,
Adair,51,4.0,475.0,0.0,12.0,3.0,2.0,73.0,0,70.0,70.0,5,0,14.600,14.000000
Adams,35,3.0,239.0,2.0,2.0,2.0,2.0,38.0,0,36.0,36.0,5,0,7.600,7.200000
Allamakee,90,7.0,835.0,11.0,26.0,17.0,12.0,131.0,0,114.0,114.0,11,0,11.909,10.363636
Appanoose,75,6.0,869.0,0.0,7.0,2.0,4.0,139.0,0,137.0,137.0,12,0,11.583,11.416667
Audubon,30,3.0,492.0,2.0,3.0,2.0,4.0,75.0,0,73.0,73.0,2,0,37.500,36.500000
Benton,200,15.0,1740.0,11.0,32.0,18.0,20.0,271.0,0,253.0,253.0,19,0,14.263,13.315789
Black Hawk,500,101.0,10119.0,132.0,294.0,188.0,174.0,1543.0,0,1355.0,1389.0,62,0,24.887,22.403226
Boone,100,19.0,2349.0,30.0,73.0,45.0,41.0,361.0,0,316.0,316.0,15,0,24.067,21.066667
Bremer,75,17.0,1721.0,9.0,11.0,11.0,7.0,261.0,0,250.0,265.0,13,0,20.077,20.384615
